In [38]:
import githubwebstats, query_ads
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests

In [39]:
github_urls = githubwebstats._clean_github_urls()

In [54]:
results = pd.DataFrame(columns=['github_url', 'repo_name', 'mentions', 'n_unq_authors', 'n_unq_first_authors',
                                'arxiv_ids', 'citation_count', 'nasa_ack_mentions', 'unq_keywords'])

for idx in tqdm(range(len(github_urls))):
    github_url = github_urls[idx]
    if requests.get('https://{}'.format(github_url)).status_code != 200:
        continue
    repo_name = github_url.split('/')[-1]
    if len(repo_name) < 4:
        continue

    r = query_ads.query_ads("full:' " + github_url +" '")

    results.loc[idx, 'github_url'] = github_url
    results.loc[idx, 'repo_name'] = repo_name
    results.loc[idx, 'mentions'] = len(r)
    results.loc[idx, 'n_unq_authors'] = len(np.unique([item for sublist in [r1['author'] for r1 in r if r1['author'] is not None] for item in sublist]))
    results.loc[idx, 'n_unq_first_authors'] = len(np.unique([r1['first_author'] for r1 in r if r1['first_author'] is not None]))


    results.loc[idx, 'unq_keywords'] = np.unique([item for sublist in [r1['keyword'] for r1 in r if r1['keyword'] is not None] for item in sublist])
    results.loc[idx, 'citation_count'] = np.sum([r1['citation_count'] for r1 in r if r1['citation_count'] is not None])
    results.loc[idx, 'nasa_ack_mentions'] = np.asarray([np.any(['NASA' in r1['ack'], 'NNX' in r1['ack']])  for r1 in r if r1['ack'] is not None]).sum()
    results.loc[idx, 'arxiv_ids'] = [r1['arxiv_id'] for r1 in r if r1['arxiv_id'] is not None]
results.to_csv('repo_success_metrics.csv', index=False)    

  0%|          | 0/855 [00:00<?, ?it/s]/Users/ch/miniconda3/lib/python3.6/site-packages/ads/search.py:544: UserWarning: Response rows did not match input rows. Setting this query's rows to 2000
  "Setting this query's rows to {}".format(self.query['rows']))
100%|██████████| 855/855 [19:43<00:00,  1.38s/it]


In [42]:
results

,github_url,repo_name,mentions,n_unq_authors,n_unq_first_authors,arxiv_ids,citation_count,nasa_ack_mentions,unq_keywords
0,github.com/aarchiba/kuiper,kuiper,1,2,1,[arXiv:1610.06545],18,0,[Statistics - Machine Learning]
1,github.com/aaronparsons/aipy,aipy,4,83,4,"[arXiv:1602.02635, arXiv:1304.4991, arXiv:1903...",350,1,[Astrophysics - Cosmology and Nongalactic Astr...
2,github.com/aburrell/aacgmv2,aacgmv2,1,10,1,[arXiv:1901.00143],0,1,[Astrophysics - Instrumentation and Methods fo...
3,github.com/aburrell/apexpy,apexpy,1,10,1,[arXiv:1901.00143],0,1,[Astrophysics - Instrumentation and Methods fo...
4,github.com/aburrell/ocbpy,ocbpy,1,10,1,[arXiv:1901.00143],0,1,[Astrophysics - Instrumentation and Methods fo...
5,github.com/acbecker/hotpants,hotpants,7,99,7,"[arXiv:1802.01692, arXiv:1809.02755, arXiv:180...",22,5,"[1988Z, Astrophysics - Astrophysics of Galaxie..."
7,github.com/achael/eht-imaging,eht-imaging,11,285,8,"[arXiv:1605.06156, arXiv:1610.05326, arXiv:190...",126,3,"[Astrophysics - Astrophysics of Galaxies, Astr..."
9,github.com/actcollaboration/zcluster,zcluster,1,44,1,[arXiv:1709.05600],31,1,[Astrophysics - Cosmology and Nongalactic Astr...
10,github.com/adamamiller/iptf16abc,iptf16abc,1,22,1,[arXiv:1708.07124],19,1,[Astrophysics - High Energy Astrophysical Phen...
11,github.com/adamginsburg/astroquery-paper,astroquery-paper,0,0,0,[],0,0,[]


In [50]:
bad_repo_names = ['kuiper', 'coronagraph', 'metal',
                  'mercury', 'teff_rad_mass_lum', 'icarus',
                  'wfc3', 'magic', 'stan','skirt','terra', 'cloudy_tables',
                  'gaia_tools','model-shift', 'beams','precession','vulcan','halogen',
                  'dust','fast','wfirst','factor','prefactor','surprise','helios','hugs',
                 'dustmaps','cepheids','codes','lira','dust_emissivity','lime','transit','inclinations',
                 'panacea','ophiuchus','atmosphere','candid','profiler','rings','validation','venice','gj876',
                 'forecaster','scarlet','starry','lemon','jitter','dusty','nebular','gj9827','aldebaran',
                  'iris','apjtemplate','slit','dave','rate','made']

In [52]:
results = results[[r.repo_name not in bad_repo_names for idx, r in results.iterrows()]]

In [53]:
results

[[False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True

In [30]:
results[results.repo_name == 'coronagraph']

,github_url,repo_name,metions,n_unq_authors,n_unq_first_authors,arxiv_ids,citation_count,nasa_ack_mentions,unq_keywords
173,github.com/jlustigy/coronagraph,coronagraph,2000,4414,995,"[arXiv:0905.3829, arXiv:1103.3275, arXiv:0911....",19083,105,[0328 ATMOSPHERIC COMPOSITION AND STRUCTURE Ex...


NameError: name 'plt' is not defined